CREATING SINGLE DATASET

In [21]:
import os
import shutil
from glob import glob


In [22]:
def update_labels(label_file, class_mapping):
    """
    Update the class labels in the label file by using a class mapping.
    
    Args:
        label_file (str): Path to the label file.
        class_mapping (dict): Dictionary mapping original class labels to new ones.
    
    Returns:
        str: Updated content of the label file.
    """
    updated_content = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_label = class_mapping[int(parts[0])]
            updated_content.append(f"{class_label} " + " ".join(parts[1:]))
    
    return "\n".join(updated_content)

In [23]:
def generate_yaml_file(output_dir, num_classes, class_names):
    """
    Create a YAML configuration file for the combined dataset.
    
    Args:
        output_dir (str): Output directory path for the combined dataset.
        num_classes (int): Total number of classes in the combined dataset.
        class_names (list): List of class names.
    """
    yaml_content = f"""
# Combined Dataset Configuration
path: {output_dir}
train: {output_dir}/train/images
val: {output_dir}/val/images
test: {output_dir}/test/images

# Number of classes
nc: {num_classes}

# Class names
names: [{', '.join([f"'{name}'" for name in class_names])}]
"""
    yaml_path = os.path.join(output_dir, "dataset_config.yaml")
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(yaml_content)
    print(f"YAML configuration file created at: {yaml_path}")

In [24]:
def combine_datasets(base_dir, dataset_dirs, num_classes_list, class_names, output_dir):
    current_max_class = 0
    global_image_counter = 1  
    for folder in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, folder, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, folder, 'labels'), exist_ok=True)
    
    for idx, dataset in enumerate(dataset_dirs):
        print(f"Processing dataset: {dataset}")
        num_classes = num_classes_list[idx]
        class_mapping = {i: i + current_max_class for i in range(num_classes)}

        for folder in ['train', 'val', 'test']:
            image_dir = os.path.join(base_dir, dataset, folder, 'images')
            label_dir = os.path.join(base_dir, dataset, folder, 'labels')
            if not os.path.exists(image_dir) or not os.path.exists(label_dir):
                print(f"Skipping {folder} folder for {dataset} as it does not exist.")
                continue

            for image_path in glob(os.path.join(image_dir, '*')):
                unique_image_name = f"{global_image_counter}.jpg"
                dest_image_path = os.path.join(output_dir, folder, 'images', unique_image_name)
                shutil.copy(image_path, dest_image_path)
                label_path = os.path.join(label_dir, os.path.splitext(os.path.basename(image_path))[0] + '.txt')
                if os.path.exists(label_path):
                    unique_label_name = f"{global_image_counter}.txt" 
                    dest_label_path = os.path.join(output_dir, folder, 'labels', unique_label_name)
                    updated_content = update_labels(label_path, class_mapping)
                    
                    with open(dest_label_path, 'w') as f:
                        f.write(updated_content)

                global_image_counter += 1 
        current_max_class += num_classes
    generate_yaml_file(output_dir, current_max_class, class_names)

In [25]:
path=r"E:\My Research Project\CODE\DATA"

In [26]:
dataset_dirs = [r"Excavators", r"People and Ladders",r"Personal Protective Equipment",r"COCO"]

In [27]:
output_dir = r"E:\My Research Project\CODE\DataCombined"

In [28]:
num_classes_list = [3,2,14,80]
class_names = [['EXCAVATORS', 'dump truck', 'wheel loader'],['Ladder', 'Person'],['Fall-Detected', 'Gloves', 'Goggles', 'Hardhat', 'Ladder', 'Mask', 'NO-Gloves', 'NO-Goggles', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest'],['aeroplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'cow', 'cup', 'diningtable', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorbike', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'pottedplant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'sofa', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tvmonitor', 'umbrella', 'vase', 'wine glass', 'zebra']]


In [29]:
combine_datasets(path, dataset_dirs, num_classes_list, class_names, output_dir)
print("Datasets combined successfully.")

Processing dataset: Excavators
Processing dataset: People and Ladders
Processing dataset: Personal Protective Equipment
Processing dataset: COCO
Skipping val folder for COCO as it does not exist.
YAML configuration file created at: E:\My Research Project\CODE\DataCombined\dataset_config.yaml
Datasets combined successfully.
